In [ ]:
import os
import cv2
import torch
import pandas as pd
import matplotlib.pyplot as plt

from metrics import Metric
from utils import process_depth_map

In [ ]:
# model_type = "DPT_Large"     # MiDaS v3 - Large     (highest accuracy, slowest inference speed)
model_type = "DPT_Hybrid"   # MiDaS v3 - Hybrid    (medium accuracy, medium inference speed)
#model_type = "MiDaS_small"  # MiDaS v2.1 - Small   (lowest accuracy, highest inference speed)

midas = torch.hub.load("intel-isl/MiDaS", model_type)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas.eval()

midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")

if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

In [ ]:
filename1 = '00019_00183_indoors_000_010.png'
filename2 = 'dog.jpg'
img1 = cv2.imread(filename1)
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
img2 = cv2.imread(filename2)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

input_batch1 = transform(img1).to(device)
input_batch2 = transform(img2).to(device)
input_batch1.shape, img1.shape, input_batch2.shape, img2.shape

In [ ]:
df = pd.read_csv('data_df.csv')
data = df[260:].reset_index(drop="true")
print(data.shape)
data.head()

In [ ]:
def general_normalize(data, origin_range, target_range):
    ori_min, ori_max = origin_range
    tar_min, tar_max = target_range
    return tar_min + (data - ori_min) * ((tar_max - tar_min) / (ori_max - ori_min))

In [ ]:
evaluation_metrics = {'img_path': [], 'mean_absrel': [], 'mean_sqrel': [], 'rms': [], 'rms_log': [], 'delta1': [], 'delta2': [], 'delta3': []}
with torch.no_grad():
    for idx, row in data.iterrows():
        image_ = cv2.imread(row['image'])
        image_ = cv2.cvtColor(image_, cv2.COLOR_BGR2RGB)
        input_image = transform(image_).to(device)
        h, w = input_image.shape[-2:]

        depth_map = process_depth_map(row['depth'], row['mask'], (w,h))
        
        pred = midas(input_image)
        pred_np = pred.permute(1, 2, 0).numpy()
        norm_pred = general_normalize(pred_np, (pred_np.min(), pred_np.max()), (depth_map.min(), depth_map.max()))
        
        metric = Metric(depth_map, norm_pred)
        evaluation_metrics['img_path'].append(row['image'])
        evaluation_metrics['mean_absrel'].append(metric.mean_absrel_err())
        evaluation_metrics['mean_sqrel'].append(metric.mean_sqrel_err())
        evaluation_metrics['rms'].append(metric.rms_err())
        evaluation_metrics['rms_log'].append(metric.rms_log_err())
        evaluation_metrics['delta1'].append(metric.percentage_of_pixel_acc()['delta1'])
        evaluation_metrics['delta2'].append(metric.percentage_of_pixel_acc()['delta2'])
        evaluation_metrics['delta3'].append(metric.percentage_of_pixel_acc()['delta3'])

In [ ]:
res_df = pd.DataFrame(evaluation_metrics)
res_df.head()

In [ ]:
res_df.to_csv('metric_midas.csv', index=False)

In [ ]:
imgs, depth_gts, depth_preds = [], [], []
with torch.no_grad():
    for idx, row in data.iterrows():
        if idx >= 4:
            break
        image_ = cv2.imread(row['image'])
        image_ = cv2.cvtColor(image_, cv2.COLOR_BGR2RGB)
        input_image = transform(image_).to(device)
        h, w = input_image.shape[-2:]

        depth_map = process_depth_map(row['depth'], row['mask'], (w,h))
        
        pred = midas(input_image)
        pred_np = pred.permute(1, 2, 0).numpy()
        
        imgs.append(image_)
        depth_gts.append(depth_map)
        depth_preds.append(pred_np)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(18, 18))

i = 0
ax[0].imshow(imgs[i])
ax[1].imshow(depth_gts[i])
ax[2].imshow(depth_preds[i])

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(18, 18))

i = 0
ax[0].imshow(imgs[i])
ax[1].imshow(depth_gts[i])
ax[2].imshow(tmp)